# This notebook is a demo version.
# Therefore, the output results in the cells below may not be reproducible.

In [1]:
# Imports
from pdf2image import convert_from_path
from pdf2image.exceptions import ( PDFInfoNotInstalledError,  PDFPageCountError,  PDFSyntaxError)
from pdfminer.high_level import extract_text
import base64
import io
import os
import concurrent.futures
from tqdm import tqdm
# from openai import OpenAI
import re
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
from rich import print as richprint
from ast import literal_eval
import unicodedata
import difflib

In [2]:
from pdfminer.high_level import extract_text
from tqdm import tqdm
import re


def make_buffer_page(page):
    new_page = page[:]
    for i in range(len(page)):
        if not page[i]:
            left = page[i - 1] if i > 0 else False
            right = page[i + 1] if i < len(page) - 1 else False
            if left or right:
                new_page[i] = True
    return new_page



def pdf_Rf_page_img(pdf_list):
    Rf_in_multi_paper = []
    for i, pdf in enumerate(pdf_list):
        print(f'pdf {i}')
        text = extract_text(pdf)
        if '\x0c' in text:

            rf_pages_bool = [('Rf' in  txt_page or 'RF' in txt_page) for txt_page in text.split('\x0c')]
            rf_pages_bool_buffer = make_buffer_page(rf_pages_bool)
            rf_pages_num = np.arange(0,len(rf_pages_bool),1)[rf_pages_bool_buffer]

            pages_text = [txt_page for txt_page in text.split('\x0c')]
            rf_pages_text = [pages_text[i] for i in rf_pages_num]
            rf_pages_united_string = '\x0c'.join(rf_pages_text)

            sen_rf_mnch_in_pdf=[]
            print("Checking each page...")
            for txt in tqdm(rf_pages_text):
                sen_list = txt.split('\n')
                sen_list = [sen for sen in sen_list if not re.fullmatch(r'\s*', sen)]
                sen_idx = [[idx-1, idx, idx+1, idx+2] for idx, sen in enumerate(sen_list) if ('Rf' in  sen or 'RF' in sen)]
                sen_rf_mnch_in_page=[]
                for mnch in sen_idx:
                    sen_rf_mnch_in_page.append([sen_list[i] for i in mnch if (i>=0)&(i<len(sen_list))])
                sen_rf_mnch_in_pdf.append(sen_rf_mnch_in_page)
                
            
        else:
            print(f'{pdf} : 페이지 표시자(\x0c)가 없습니다!!')
            continue

        imgs = convert_from_path( pdf, poppler_path = '/home/ufslab223/anaconda3/envs/ksw/bin')
        rf_page_imgs = [imgs[i] for i in rf_pages_num]
        Rf_in_multi_paper.append(rf_page_imgs)
    return Rf_in_multi_paper, rf_pages_united_string, rf_pages_text, sen_rf_mnch_in_pdf



# Converting images to base64 encoded images in a data URI format to use with the ChatCompletions API
# 이미지 이진 데이터-->텍스트 문자열 (base64)
# URI : 파일 내용을 문자열로 표현한 URL 형식

def get_img_uri(img):
    png_buffer = io.BytesIO()
    img.save(png_buffer, format="PNG")
    png_buffer.seek(0)

    base64_png = base64.b64encode(png_buffer.read()).decode('utf-8')

    data_uri = f"data:image/png;base64,{base64_png}"
    return data_uri

def get_img_markdown(img):
    
    uri = get_img_uri(img)
    markdown = f'![image](data:image/png;base64,{uri})\n\nDescribe this image.'

# GPT-API (one page each)

### opne ai api key 발급 및 결제 필요

In [ ]:
print('This section is not included here because the research has not been finalized yet.\nMore details will be shared after publication.\nFor inquiries, please contact the author.')

#GPT-API
import openai
from openai import OpenAI
import time

def safe_chat(client, **kwargs):
    # 분당 토큰 처리 일정 이상 넘어가면 에러 뜨는데, 몇초 기다리면 제대로 됨. 
    # 중간에 안터지게 몇초 기다리고 재시도 하게 하는 함수
    for _ in range(5): # 5변 시도
        try:
            return client.chat.completions.create(**kwargs)
        except openai.RateLimitError as e:
            print(" 분당 토큰 수 제한 때문에... 잠깐만 기달려봐...")
            time.sleep(1.5) # 1.5초 기다렸다가 재시도
    raise RuntimeError("분당 토큰 수 제한이 계속 걸리네... 나중에 다시 시도해봐")


def analyze_one_page_GPT(one_page_text):
    # Open AI api key를 발급받아서 입력하세요.
    client = OpenAI(api_key='')

    json_style_example='''
    [
      {
        "compound name": "26b",
        "compound IUPAC name": "the given IUPAC name",
        "eluent1": "PE",
        "eluent2": "EA",
        "ratio": "20/1",
        "Rf": "0.6"
      },
      {
        "compound name": "26c",
        "compound IUPAC name": "another IUPAC name",
        "eluent1": "CH2Cl2",
        "eluent2": "np.nan",
        "ratio": "1",
        "Rf": "0.4"
      }
    ]
    '''
    print('GPT started to analyze pdf')
    answer=[]
    for i in tqdm(range(len(one_page_text))):
        text_input = [{"type": "text", "text": one_page_text[i]}]
        #image_input = [{"type": "image_url", "image_url": {"url": one_page_image[i]}}]

        if i==0:
            supporting_page_text = "This is the first page, so there is no previous page."
        else:
            supporting_page_text = one_page_text[i-1]

        if len(answer)>0:
            clean_answer = answer[-1].strip().removeprefix("```json").removesuffix("```").strip()
            previous_answer = json.loads(clean_answer)
        else:
            previous_answer=[]

        system_prompt = f'''

        This section is not included here because the research has not been finalized yet.
        More details will be shared after publication.
        For inquiries, please contact the author.
        
        '''
        
        response = safe_chat(client,
                             model="gpt-4.1-mini",
                             messages=[
                                 {"role": "system", "content": system_prompt},
                                 {"role": "user", "content": text_input}# + image_input},
                             ],
                             temperature=0,
                             top_p=0.1,
                             #reasoning_effort='high'
                            )
        answer.append(response.choices[0].message.content)
        
    return answer


# collect Rf pages

In [4]:
def collect_rf_pages(pdf_file):
    print('preprocessing pdf')
    # pdf_file example : 'ol4c04571_si_001.pdf'

    # extract
    path = './SI/'
    pdf_list= [path+pdf_file]
    rf_multi_paper, rf_multi_string, short_text, sen_rf_mnch_in_pdf = pdf_Rf_page_img(pdf_list)

    #image --> url
    rf_url_multi_paper = []
    for rf_one_paper in rf_multi_paper:
        rf_page_url = [get_img_uri(rf_page) for rf_page in rf_one_paper]
        rf_url_multi_paper.append(rf_page_url)
        
    return rf_multi_string, short_text,  sen_rf_mnch_in_pdf, rf_url_multi_paper

# extract data (GPT)

In [5]:
def normalize_ligatures(s):
    return unicodedata.normalize('NFKC', s)

def most_similar_substring(text, target, mode = 'IUPAC_string', starting_index = 0):
    text =  normalize_ligatures(re.sub(r'\s+','',text))
    best_match = ""
    highest_ratio = 0
    best_index = -1
    if type(target)==str:
        target_len = len(target)
        for i in range(starting_index, len(text) - target_len + 1):
            if (i + target_len)>len(text):
                print('check IUPAC. similar IUPAC is out of range')
                best_index = len(text)
                break
            chunk = text[i:i+target_len]
            ratio = difflib.SequenceMatcher(None, chunk, target).ratio()
            if ratio > highest_ratio:
                highest_ratio = ratio
                best_match = chunk
                best_index = i
    if mode == 'IUPAC_string':            
        return best_match.strip()
    elif mode == 'IUPAC_index':
        return best_index

def most_similar_substring_limited_idx(text, target, ending_index, mode = 'IUPAC_string',  starting_index = 0):
    text =  normalize_ligatures(re.sub(r'\s+','',text))
    best_match = ""
    highest_ratio = 0
    if type(target)==str:
        target_len = len(target)
        for i in range(starting_index, ending_index):
            if (i + target_len)>len(text):
                print('check IUPAC. similar IUPAC is out of range')
                best_index = len(text)
                break
            chunk = text[i:i+target_len]
            ratio = difflib.SequenceMatcher(None, chunk, target).ratio()
            if ratio > highest_ratio:
                highest_ratio = ratio
                best_match = chunk
                best_index = i
    if mode == 'IUPAC_string':            
        return best_match.strip()
    elif mode == 'IUPAC_index':
        return best_index


def false_iupac_compare(short_text, data_final, false_page_idx):
    page_idx_list = list(false_page_idx.items())
    for pi_pair in page_idx_list:
        page_text = short_text[pi_pair[0]]
        if pi_pair[0] > 0:
            pre_page_text = short_text[pi_pair[0]-1]
        iupac_list = data_final.loc[pi_pair[1], 'compound IUPAC name'].tolist()
        for i, iupac in enumerate(iupac_list):
            iupac_in_paper = most_similar_substring(page_text, iupac)
            richprint(f'[bold red]index___{pi_pair[1][i]}[/bold red]')
            richprint(f'[bold red]name: {data_final.loc[pi_pair[1][i], 'compound name']}[/bold red]')
            if type(iupac)==str:
                print('LLM_output >>>>  ', re.sub(r'\s+','',iupac))
            else:
                print('LLM_output >>>>  ', iupac)
            print('TXT_current >>>  ', iupac_in_paper)
            if pi_pair[0] > 0:
                iupac_in_paper_pre = most_similar_substring(pre_page_text, iupac)
                print('TXT_previous >>  ', iupac_in_paper_pre,'\n TXT_current가 전혀 비슷하지 않을때 참고')
            

def extract_data_GPT(pdf_file):
    import json
    whole_string, short_text, sen_rf_mnch_in_pdf, image_url_list = collect_rf_pages(pdf_file)
    image = image_url_list[0]
    if len(short_text)<=1:
        print('Cannot detect Rf pages!!!')
        return None
    extract_output1 = analyze_one_page_GPT(one_page_text = short_text)
    false_page_idx = {}
    whole_data=[]
    df_idx_start=0
    for idx, data in enumerate(extract_output1):
        clean_data = data.strip().removeprefix("```json").removesuffix("```").strip()
        dict_data = json.loads(clean_data)

        # data double check
        
        # --> Rf data check
        
        if len(dict_data)==0:
            print('No Rf data in this page')
        else:
            df_data = pd.DataFrame(dict_data)
            richprint(f'[bold bright_red]Page ---#################################---{idx}[/bold bright_red]')
            df = df_data.loc[:,['compound name','eluent1','eluent2','ratio','Rf']]
            num = len(df)
            df.index = np.arange(df_idx_start, df_idx_start + num, 1)
            df_idx_start += num
            display(pd.DataFrame(df).loc[:,['compound name','eluent1','eluent2','ratio','Rf']])


            # --> iupac name & matching with Rf data check

            short_text_flat = normalize_ligatures(re.sub(r'\s+','',short_text[idx]).lower())
            page_iupac_index = [short_text_flat.find(re.sub(r'\s+','',normalize_ligatures(str(iupac))).lower()) for iupac in df_data['compound IUPAC name']]
            
            for_check = page_iupac_index.copy()
            if (for_check[0] == -1)&(idx>0):
                previous_short_text_flat = normalize_ligatures(re.sub(r'\s+','',short_text[idx-1]).lower())
                pre_page_iupac_index = [previous_short_text_flat.find(re.sub(r'\s+','',normalize_ligatures(str(iupac))).lower()) for iupac in df_data['compound IUPAC name']]
                where = np.where(np.array(pre_page_iupac_index)!= -1)[0]
                for i in where:
                    max_idx = len(previous_short_text_flat)
                    minus_index = pre_page_iupac_index[i]-(max_idx+100)
                    for_check[i] = minus_index
                    page_iupac_index[i] = minus_index
                
            if -1 in for_check:
                minus_one_idx = np.where(np.array(for_check)==-1)[0]
                false_index = minus_one_idx + (df_idx_start-num)
                print(f'wrong iupac in this page:{false_index}')
                #print(num)
                false_page_idx[idx] = false_index

                for i in minus_one_idx:
                    query_iupac = df_data.loc[i,'compound IUPAC name']

                    left = 0
                    right = len(short_text_flat) - len(str(query_iupac)) + 1

                    left_candidates = [idx_val for idx_val in for_check[:i] if idx_val != -1]
                    if left_candidates:
                        left = max(left_candidates)

                    right_candidates = [idx_val for idx_val in for_check[i+1:] if idx_val != -1]
                    if right_candidates:
                        right = min(right_candidates) - len(str(query_iupac))

                    left = max(left, 0)
                    right = max(right, left + 1)

                    iupac_similar_index = most_similar_substring_limited_idx( short_text_flat,query_iupac,ending_index=right,mode='IUPAC_index',starting_index=left)

                    for_check[i] = iupac_similar_index



             ######################################## 여기서 부터 본격적으로 IUPAC~Rf 매칭 검증 
            # 위에서 page_iupac_index : 이전 페이지 등장하는 IUPAC 고려한 인덱스
            # 위에서 for_check : 이전 페이지 등장 IUPAC 고려 + 틀리게 뽑은 IUPAC은 가장 유사한 string 인덱스로 데체

            # Rf indexing
            short_text_flat_2=  normalize_ligatures(re.sub(r'\s+','',short_text[idx])) # same 'short_text_flat' but no 'lower()'  
            previous_short_text_flat_2 = normalize_ligatures(re.sub(r'\s+','',short_text[idx-1])) # same 'previous short_text_flat' but no 'lower()'  
            pre_full_len = len(previous_short_text_flat_2)
            Rf_index = np.array([i for i in range(len(short_text_flat_2) - 1) if (short_text_flat_2[i:i+2] == 'Rf') or (short_text_flat_2[i:i+2] == 'RF')])

            # IUPAC , Rf 등장 순서 재현하여 출력
            sen_rf_mnchs_list = sen_rf_mnch_in_pdf[idx]
            # 페이지에 데이터 1개일때
            if len(for_check) == 1:
                iupac_idx = for_check[0]
                Rf_location_up = Rf_index[Rf_index < iupac_idx]
                Rf_location_down = Rf_index[Rf_index > iupac_idx]

                if len(Rf_location_up) > 0:
                    richprint(f'[bold green1]***Rf_position: {Rf_location_up}***[/bold green1]')
                    mask = np.isin(Rf_index, Rf_location_up)
                    sen_rf_mnchs = [m for n, m in zip(mask, sen_rf_mnchs_list) if n]
                    for mnch in sen_rf_mnchs:
                        mnch_txt = '\n'.join(mnch)
                        print(mnch_txt + '\n')

                
                iupac_length = len(df_data.loc[0, 'compound IUPAC name']) if pd.notna(df_data.loc[0,'compound IUPAC name']) else 0
                if iupac_idx < 0:
                    org_idx = iupac_idx + (pre_full_len + 100)
                    iupac_label = previous_short_text_flat_2[org_idx:(org_idx+iupac_length)]
                else :
                    iupac_label = short_text_flat_2[iupac_idx:(iupac_idx+iupac_length)]
                richprint(f'[bold orange1]IUPAC position: {iupac_idx} --> {iupac_label}[/bold orange1]')

                if len(Rf_location_down) > 0:
                    richprint(f'[bold green1]***Rf_position: {Rf_location_down}***[/bold green1]')
                    mask = np.isin(Rf_index, Rf_location_down)
                    sen_rf_mnchs = [m for n, m in zip(mask, sen_rf_mnchs_list) if n]
                    for mnch in sen_rf_mnchs:
                        mnch_txt = '\n'.join(mnch)
                        print(mnch_txt + '\n')


            else:
                # 페이지에 데이터 2개 이상
                for i, iupac_idx in enumerate(for_check):
                    if i ==0:
                        Rf_location_up = Rf_index[Rf_index < iupac_idx]
                        Rf_location_down = Rf_index[(Rf_index > iupac_idx) & (Rf_index < for_check[1])]
                        if len(Rf_location_up)>0:
                            richprint(f'[bold green1]***Rf_position: {Rf_location_up}***[/bold green1]')
                            mask = np.isin(Rf_index, Rf_location_up)
                            sen_rf_mnchs = [m for n,m in zip(mask, sen_rf_mnchs_list) if n]
                            for mnch in sen_rf_mnchs:
                                mnch_txt = '\n'.join(mnch)
                                print(mnch_txt+'\n')

                        iupac_length = len(df_data.loc[i, 'compound IUPAC name']) if pd.notna(df_data.loc[i,'compound IUPAC name']) else 0
                        if iupac_idx < 0:
                            org_idx = iupac_idx + (pre_full_len + 100)
                            iupac_label = previous_short_text_flat_2[org_idx:(org_idx+iupac_length)]
                        else :
                            iupac_label = short_text_flat_2[iupac_idx:(iupac_idx+iupac_length)]
                        richprint(f'[bold orange1]IUPAC position: {iupac_idx} --> {iupac_label}[/bold orange1]')

                        if len(Rf_location_down)>0:
                            richprint(f'[bold green1]***Rf_position: {Rf_location_down}***[/bold green1]')
                            mask = np.isin(Rf_index, Rf_location_down)
                            sen_rf_mnchs = [m for n,m in zip(mask, sen_rf_mnchs_list) if n]
                            for mnch in sen_rf_mnchs:
                                mnch_txt = '\n'.join(mnch)
                                print(mnch_txt+'\n')
                    elif i == len(for_check)-1 :
                        Rf_location_down = Rf_index[Rf_index > iupac_idx]

                        iupac_length = len(df_data.loc[i, 'compound IUPAC name']) if pd.notna(df_data.loc[i,'compound IUPAC name']) else 0
                        if iupac_idx < 0:
                            org_idx = iupac_idx + (pre_full_len + 100)
                            iupac_label = previous_short_text_flat_2[org_idx:(org_idx+iupac_length)]
                        else :
                            iupac_label = short_text_flat_2[iupac_idx:(iupac_idx+iupac_length)]
                        richprint(f'[bold orange1]IUPAC position: {iupac_idx} --> {iupac_label}[/bold orange1]')
                            
                        if len(Rf_location_down)>0:
                            richprint(f'[bold green1]***Rf_position: {Rf_location_down}***[/bold green1]')
                            mask = np.isin(Rf_index, Rf_location_down)
                            sen_rf_mnchs = [m for n,m in zip(mask, sen_rf_mnchs_list) if n]
                            for mnch in sen_rf_mnchs:
                                mnch_txt = '\n'.join(mnch)
                                print(mnch_txt+'\n')
                    else:
                        Rf_location_down = Rf_index[(Rf_index > iupac_idx) & (Rf_index < for_check[i+1])]

                        iupac_length = len(df_data.loc[i, 'compound IUPAC name']) if pd.notna(df_data.loc[i,'compound IUPAC name']) else 0
                        if iupac_idx < 0:
                            org_idx = iupac_idx + (pre_full_len + 100)
                            iupac_label = previous_short_text_flat_2[org_idx:(org_idx+iupac_length)]
                        else :
                            iupac_label = short_text_flat_2[iupac_idx:(iupac_idx+iupac_length)]
                        richprint(f'[bold orange1]IUPAC position: {iupac_idx} --> {iupac_label}[/bold orange1]')
                        
                        if len(Rf_location_down)>0:
                            richprint(f'[bold green1]***Rf_position: {Rf_location_down}***[/bold green1]')
                            mask = np.isin(Rf_index, Rf_location_down)
                            sen_rf_mnchs = [m for n,m in zip(mask, sen_rf_mnchs_list) if n]
                            for mnch in sen_rf_mnchs:
                                mnch_txt = '\n'.join(mnch)
                                print(mnch_txt+'\n')
                                
            ########################################## 여기까지가 IUPAC, RF 매칭 검증 (등장 순서 재현)
        
        whole_data += dict_data
        
    for datapoint in whole_data:
        for key, value in datapoint.items():
            if value == "np.nan":
                datapoint[key] = np.nan
    
    data_final = pd.DataFrame(whole_data)

    if len(data_final)==0:
        print("No data")
        return None
    else:
        # iupac name check

        if len(false_page_idx)==0:
            print('Nice')
            print('Every IUPAC is correct')
            
        else:
            richprint('Wrong IUPAC names')
            richprint(false_page_idx)

            false_iupac_compare(short_text, data_final, false_page_idx)

            

        return data_final, false_page_idx

# one touch extract

In [ ]:
# start!

# 이번주 는 datatable_123 부터 카운트 

title='dmmy title'
doi='dmmy doi'
publish='YYYY.vol.issue'

In [ ]:
data_final, false_page_idx = extract_data_GPT('1.pdf')

preprocessing pdf
pdf 0
Checking each page...


100%|██████████| 24/24 [00:00<00:00, 16808.03it/s]


GPT started to analyze pdf


100%|██████████| 24/24 [01:12<00:00,  3.02s/it]

No Rf data in this page


...

***Rf_position: [302]***

(10  to  20%  EtOAc/Hexane)  to  give  9-membered  lactone  compound  (18)  as  a  white  solid  after 
recrystallization from n-hexane/DCM (3:1) (0.10 g, 0.33 mmol, 55% yield); Rf = 0.1 (EtOAc/Hex 2:8). M. 
Pt. 135 °C.  1H NMR (400 MHz, CDCl3): δ 7.16 (d, J = 8.4 Hz, 2H), 6.83 (d, J = 8.8 Hz, 2H), 5.61 (dt, J = 
14.0, 6.8 Hz, 1H), 5.41 – 5.36 (m, 2H), 4.21 (dt, J = 11.2, 5.6 Hz, 1H), 4.14 – 4.08 (m, 1H), 3.77 (s, 3H), 



IUPAC position: 798 --> 2-hydroxybut-3-en-1-ylbenzoate(

***Rf_position: [1145]***

product was isolated by preparative TLC (10% EtOAc/Hex, 3 times run) to give the desired product 2-
hydroxybut-3-en-1-yl benzoate 5aa (32.6 mg, 85% yield) as transparent oil. Rf = 0.2 (EtOAc/Hex 1:9). 
1H NMR (500 MHz, CDCl3) δ 8.09 – 7.96 (m, 2H), 7.58 – 7.48 (m, 1H), 7.41 (dd, J = 14.0, 4.8 Hz, 2H), 
5.92 (ddd, J = 16.5, 10.5, 5.5 Hz, 1H), 5.42 (dt, J = 17.5, 1.5 Hz, 1H), 5.25 (dt, J = 11.0, 1.5 Hz, 1H), 4.55 

No Rf data in this page
Nice
Every IUPAC is correct


In [186]:
print(f'총 개수: {len(data_final)}개')

총 개수: 49개


In [187]:
# delete wrong

del_list=[]
for arr in list(false_page_idx.values()):
    del_list += list(arr)
del_list

del_method = input("Do you want to use list? (y/n): ")
if del_method == 'y':
    user_input = input("삭제 리스트에서 제외할 옳은 인덱스들 입력 (띄어쓰기로 구분): ")
    if user_input == "":
        data_final.drop(del_list,axis=0, inplace=True)
    else:
        exe = [float(x) for x in user_input.split()]
        for i in exe:
            del_list.remove(i)
            data_final.drop(del_list,axis=0, inplace=True)
    print(f'{del_list} is deleted: length = {len(data_final)}')
elif del_method == 'n': 
    user_input = input("틀린 인덱스 입력 (띄어쓰기로 구분): ")
    numbers = [float(x) for x in user_input.split()]
    data_final.drop(numbers, axis=0, inplace=True)
    print(f'{numbers} is in delete list: length = {len(data_final)}')



# data_final.index = range(len(data_final))

[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 18.0, 19.0, 20.0, 21.0, 37.0, 38.0, 39.0, 40.0] is in delete list: length = 34


In [188]:
data_final

,compound name,compound IUPAC name,eluent1,eluent2,ratio,Rf
0,1c-A,Ethyl 2-cyclopentylideneacetate,EtOAc,Hex,0.5/9.5,0.62
1,1c-B,2-Cyclopentylideneethan-1-ol,EtOAc,Hex,2/8,0.32
2,1c-C,2-cyclopentylideneacetaldehyde,EtOAc,Hex,2/8,0.45
10,Ethyl 3-butylhept-2-enoate (1e-A),Ethyl 3-butylhept-2-enoate,EtOAc,Hex,0.5/9.5,0.62
11,"2,3-butylhept-2-en-1-ol (1e-B)","2,3-butylhept-2-en-1-ol",EtOAc,Hex,1/9,0.25
12,3-butylhept-2-enal (1e-C),3-butylhept-2-enal,EtOAc,Hex,2/8,0.72
13,5-allylidenenonane (1e-D),5-allylidenenonane,Hex,NaN,1,0.90
14,"2,2-dibutyl-3-vinyloxirane ((±)-1e)","2,2-dibutyl-3-vinyloxirane",EtOAc,Hex,0.5/9.5,0.42
15,1-phenylprop-2-en-1-one (1f-B),1-phenylprop-2-en-1-one,EtOAc,Hex,1/9,0.6
16,"2,3-Epoxy-1-(phenyl)-1-propanone (1f-C)","2,3-Epoxy-1-(phenyl)-1-propanone",EtOAc,Hex,1/9,0.2


In [189]:
# delete nan

data_final=data_final[np.all(~pd.isna(data_final[['compound IUPAC name','eluent1','ratio','Rf']]), axis=1)]
data_final.index = range(len(data_final))
len(data_final)

33

In [190]:
# 중복제거
data_id = ['_'.join(map(str,data_final.iloc[i,[1,2,3,4,5]]))for i in range(len(data_final))]
data_final.loc[:,'data_ID'] = data_id
data_final = data_final.groupby(by = 'data_ID', as_index = False).agg({'compound name':'first',
                                                                      'compound IUPAC name' : 'first',
                                                                      'eluent1' : 'first',
                                                                      'eluent2' : 'first',
                                                                      'ratio' : 'first',
                                                                      'Rf' : 'first'})
data_final = data_final.reset_index(drop=True)
len(data_final)

/tmp/ipykernel_3010094/496171019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final.loc[:,'data_ID'] = data_id


33

In [191]:
data_final

,data_ID,compound name,compound IUPAC name,eluent1,eluent2,ratio,Rf
0,(6S)-6-(((tert-butyldimethylsilyl)oxy)methyl)-...,12,(6S)-6-(((tert-butyldimethylsilyl)oxy)methyl)-...,DCM,Hex,1/1,0.4
1,"(R,E)-6-((tert-butyldimethylsilyl)oxy)-2-pheny...",16,"(R,E)-6-((tert-butyldimethylsilyl)oxy)-2-pheny...",EtOAc,Hex,2/8,0.21
2,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,15,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,EtOAc,Hex,1/9,0.52
3,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,13,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,EtOAc,Hex,1/9,0.44
4,(S)-1-(1-hydroxy-4-methylcyclohexyl)allyl 2-(4...,4’jh,(S)-1-(1-hydroxy-4-methylcyclohexyl)allyl 2-(4...,DCM,None,1,0.30
5,(S)-1-(1-hydroxycyclohexyl)allyl benzoate_DCM_...,4ad,(S)-1-(1-hydroxycyclohexyl)allyl benzoate,DCM,None,1,0.4
6,(S)-1-(1-hydroxycyclopentyl)allyl 2-(4-methoxy...,4’jc,(S)-1-(1-hydroxycyclopentyl)allyl 2-(4-methoxy...,DCM,None,1,0.30
7,(S)-1-(4-(tert-butyl)-1-hydroxycyclohexyl)ally...,4’jg,(S)-1-(4-(tert-butyl)-1-hydroxycyclohexyl)ally...,DCM,None,1,0.30
8,(S)-1-hydroxy-2-methylbut-3-en-2-yl 2-(4-metho...,4’jb,(S)-1-hydroxy-2-methylbut-3-en-2-yl 2-(4-metho...,DCM,None,1,0.35
9,(S)-1-hydroxy-3-phenylbut-3-en-2-yl benzoate_D...,4af,(S)-1-hydroxy-3-phenylbut-3-en-2-yl benzoate,DCM,None,1,0.38


# process

In [193]:
#much_faster

import subprocess

def iupac_to_smiles_local_opsin(iupac_name):
    jar_path = 'opsin-cli-2.8.0-jar-with-dependencies.jar'  # 본인 경로
    command = ['java', '-jar', jar_path]
    
    result = subprocess.run(command, input = iupac_name, capture_output=True, text=True)

    if result.returncode == 0:
        smiles = result.stdout.strip()
        if smiles:
            return smiles
        else:
            return None
    else:
        raise RuntimeError(f"OPSIN failed: {result.stderr}")


In [194]:
# compound: IUPAC --> smiles

smiles_list = []
for iupac in tqdm(data_final['compound IUPAC name']):
    if iupac:
        smiles_list.append(iupac_to_smiles_local_opsin(iupac))
    else:
        smiles_list.append(np.nan)
        
data_final2 = data_final.copy(deep=True)
data_final2['compound IUPAC name'] = smiles_list
data_final2.rename(columns={'compound IUPAC name': 'Smiles'}, inplace=True)
data_final2=data_final2[pd.notna(data_final2['Smiles'])]
data_final2.reset_index(drop=True, inplace=True)
data_final2

100%|██████████| 33/33 [00:16<00:00,  1.95it/s]


,data_ID,compound name,Smiles,eluent1,eluent2,ratio,Rf
0,(6S)-6-(((tert-butyldimethylsilyl)oxy)methyl)-...,12,[Si](C)(C)(C(C)(C)C)OC[C@@H]1CC=CC(O1)=O,DCM,Hex,1/1,0.4
1,"(R,E)-6-((tert-butyldimethylsilyl)oxy)-2-pheny...",16,[Si](C)(C)(C(C)(C)C)OC/C=C/C[C@@H](C(=O)O)C1=C...,EtOAc,Hex,2/8,0.21
2,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,15,C1(=CC=CC=C1)CC(=O)O[C@H](CO[Si](C)(C)C(C)(C)C...,EtOAc,Hex,1/9,0.52
3,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,13,C(C=CC1=CC=CC=C1)(=O)O[C@H](CO[Si](C)(C)C(C)(C...,EtOAc,Hex,1/9,0.44
4,(S)-1-(1-hydroxy-4-methylcyclohexyl)allyl 2-(4...,4’jh,COC1=CC=C(C=C1)CC(=O)O[C@@H](C=C)C1(CCC(CC1)C)O,DCM,None,1,0.30
5,(S)-1-(1-hydroxycyclohexyl)allyl benzoate_DCM_...,4ad,C(C1=CC=CC=C1)(=O)O[C@@H](C=C)C1(CCCCC1)O,DCM,None,1,0.4
6,(S)-1-(1-hydroxycyclopentyl)allyl 2-(4-methoxy...,4’jc,COC1=CC=C(C=C1)CC(=O)O[C@@H](C=C)C1(CCCC1)O,DCM,None,1,0.30
7,(S)-1-(4-(tert-butyl)-1-hydroxycyclohexyl)ally...,4’jg,COC1=CC=C(C=C1)CC(=O)O[C@@H](C=C)C1(CCC(CC1)C(...,DCM,None,1,0.30
8,(S)-1-hydroxy-3-phenylbut-3-en-2-yl benzoate_D...,4af,C(C1=CC=CC=C1)(=O)O[C@H](CO)C(=C)C1=CC=CC=C1,DCM,None,1,0.38
9,(S)-2-(2-(4-methoxyphenyl)acetoxy)but-3-en-1-y...,17,C(CC=C)(=O)OC[C@H](C=C)OC(CC1=CC=C(C=C1)OC)=O,EtOAc,Hex,1/9,0.625


# save

In [195]:
import os

folder_path = "data_extracted_gpt"
file_count = len([
    f for f in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, f))
])

richprint(f"📁 '{folder_path}' 폴더에 있는 파일 개수: {file_count}")

# gpt -data 개수 확인
data_num=[]
for i in np.arange(1,file_count+1,1):
    num = len(pd.read_csv(f'./data_extracted_gpt/datatable{i}.csv'))
    data_num.append(num)
richprint(f'논문 별 평균 데이터 수: {np.mean(data_num)}')
richprint(f'gpt가 모은 데이터 수: {np.sum(data_num)}')

📁 'data_extracted_gpt' 폴더에 있는 파일 개수: 131

논문 별 평균 데이터 수: 37.38931297709924

gpt가 모은 데이터 수: 4898

In [196]:
if os.path.isfile(f'data_extracted_gpt/datatable{file_count}.csv'):
    last_file = pd.read_csv(f'data_extracted_gpt/datatable{file_count}.csv')
    last_title = last_file['title'][0]
    last_doi = last_file['doi'][0]
else:
    last_doi = 'none'

In [197]:
print(last_title)
print(last_doi)

Palladium-Catalyzed and Photoinduced Benzylic C–H Carbonylation/Annulation under Mild Conditions
https://pubs.acs.org/doi/10.1021/acs.orglett.2c02877


In [198]:
print(title)
print(doi)

Iridium-Catalyzed Enantioselective Ring Opening of Alkenyl Oxiranes by Unactivated Carboxylic Acids
https://pubs.acs.org/doi/10.1021/acs.orglett.2c02919


In [199]:
if doi ==last_doi:
    print('Check doi')
else:
    data_final3= data_final2[~pd.isna(data_final2['Smiles'])].copy()
    data_final3['title'] = [title for i in range(len(data_final3))]
    data_final3['doi'] = [doi for i in range(len(data_final3))]
    data_final3['publish'] = [publish for i in range(len(data_final3))]
    data_final3.index = range(len(data_final3))
    data_final3
data_final3

,data_ID,compound name,Smiles,eluent1,eluent2,ratio,Rf,title,doi,publish
0,(6S)-6-(((tert-butyldimethylsilyl)oxy)methyl)-...,12,[Si](C)(C)(C(C)(C)C)OC[C@@H]1CC=CC(O1)=O,DCM,Hex,1/1,0.4,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
1,"(R,E)-6-((tert-butyldimethylsilyl)oxy)-2-pheny...",16,[Si](C)(C)(C(C)(C)C)OC/C=C/C[C@@H](C(=O)O)C1=C...,EtOAc,Hex,2/8,0.21,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
2,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,15,C1(=CC=CC=C1)CC(=O)O[C@H](CO[Si](C)(C)C(C)(C)C...,EtOAc,Hex,1/9,0.52,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
3,(S)-1-((tert-butyldimethylsilyl)oxy)but-3-en-2...,13,C(C=CC1=CC=CC=C1)(=O)O[C@H](CO[Si](C)(C)C(C)(C...,EtOAc,Hex,1/9,0.44,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
4,(S)-1-(1-hydroxy-4-methylcyclohexyl)allyl 2-(4...,4’jh,COC1=CC=C(C=C1)CC(=O)O[C@@H](C=C)C1(CCC(CC1)C)O,DCM,None,1,0.30,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
5,(S)-1-(1-hydroxycyclohexyl)allyl benzoate_DCM_...,4ad,C(C1=CC=CC=C1)(=O)O[C@@H](C=C)C1(CCCCC1)O,DCM,None,1,0.4,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
6,(S)-1-(1-hydroxycyclopentyl)allyl 2-(4-methoxy...,4’jc,COC1=CC=C(C=C1)CC(=O)O[C@@H](C=C)C1(CCCC1)O,DCM,None,1,0.30,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
7,(S)-1-(4-(tert-butyl)-1-hydroxycyclohexyl)ally...,4’jg,COC1=CC=C(C=C1)CC(=O)O[C@@H](C=C)C1(CCC(CC1)C(...,DCM,None,1,0.30,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
8,(S)-1-hydroxy-3-phenylbut-3-en-2-yl benzoate_D...,4af,C(C1=CC=CC=C1)(=O)O[C@H](CO)C(=C)C1=CC=CC=C1,DCM,None,1,0.38,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40
9,(S)-2-(2-(4-methoxyphenyl)acetoxy)but-3-en-1-y...,17,C(CC=C)(=O)OC[C@H](C=C)OC(CC1=CC=C(C=C1)OC)=O,EtOAc,Hex,1/9,0.625,Iridium-Catalyzed Enantioselective Ring Openin...,https://pubs.acs.org/doi/10.1021/acs.orglett.2...,2022.24.40


In [200]:
# 최종 저장
data_final3.to_csv(f'data_extracted_gpt/datatable{file_count+1}.csv', index=False)